# Generate the systems for multistates

In [2]:
#IMPORT
import os, sys, glob
import time

from collections import defaultdict
from pygromos.gromos.gromosPP import GromosPP
from pygromos.gromos.gromosXX import GromosXX
from pygromos.files import imd
from pygromos.data import solvents

from pygromos.files.coord import cnf
from pygromos.files.blocks.coord_blocks import GENBOX, Pbc
from pygromos.files.blocks import imd_blocks
from pygromos.data import imd_templates

import restraintmaker

#CHANGE HERE
gromosPP_bin_path = "/home/bschroed/Documents/code/gromosPP/installed/bin"
gromosXX_bin_path = "/home/bschroed/Documents/code/gromosXX/installed/bin"
restraintmaker_path = os.path.abspath(os.path.dirname(restraintmaker.__file__)+"/..")
emin_imd = imd_templates.template_emin

/home/bschroed/Documents/code/restraintmaker/restraintmaker/submodules/pygromos/pygromos/files/topology/top.py:20: UserWarning: Module topo-files is under Development and not entirely implemented!
  warnings.warn("Module topo-files is under Development and not entirely implemented!")


## Path definitions
generate the subfolders for the systems - no changes required here.

In [3]:
sets_dir = restraintmaker_path+"/devtools/otherScripts/b_ATB_solvationFreeEnergies/sets"
pairwise_dir = sets_dir+"/pairwise"

resn_lib_path = sets_dir+"/resn_lib.lib"


gromPP = GromosPP(gromosPP_bin_path)
gromXX = GromosXX(gromosXX_bin_path)

atb_dirs = restraintmaker_path+"/devtools/otherScripts/b_ATB_solvationFreeEnergies/ATB_molecules"
sys.path.append(atb_dirs+"/..")

##get all_single file_tops:
all_tops = glob.glob(atb_dirs+"/*/*top")
state_all_tops={os.path.basename(value).split(".")[0]: value for value in all_tops}




GROMOSPATH /home/bschroed/Documents/code/gromosXX/installed/bin/


## Build gromos files

In [4]:
import utils_test_set_ATB as utils
all_states = utils.multistate_ligand_sets["all"]
all_combos = [x for x in os.listdir(pairwise_dir) if(os.path.isdir(pairwise_dir+"/"+x))]

all_combos = [x if(len(x)==2) else (x[0], "_"+x[2]) for x in list(map(lambda x: x.split("_"), all_combos))]
print(all_combos)
print()

for state_a, state_b in all_combos:
    system_tops = [state_all_tops[x] for x in [state_a, state_b]]

    name = state_a+"_"+state_b
    out_dir = pairwise_dir+"/"+name
    out_prefix_path = out_dir+"/"+name
    print(name, [state_a, state_b])


    #build dualTop system
    out_top_path, out_ptp_path = gromPP.prep_eds(in_top_paths=system_tops, number_of_eds_states=len(system_tops), out_file_path=out_prefix_path)

    #generate cnf
    in_pdb = glob.glob(out_dir+"/*.pdb")[0]
    out_cnf_path = gromPP.pdb2gromos(in_pdb_path=in_pdb, in_top_path=out_top_path, in_lib_path=resn_lib_path,
                                     out_cnf_path=out_prefix_path+".cnf")

    ##build box
    box_cnf = cnf.Cnf(out_cnf_path)
    box_cnf.supress_atomPosition_singulrarities()
    box_cnf.GENBOX = GENBOX(pbc=Pbc.rectangular,
                            length=[3.610220118 for x in range(3)],
                            angles=[90 for x in range(3)],
                            euler=[0 for x in range(3)],
                            origin=[0 for x in range(3)])
    box_cnf.write(out_cnf_path)

    #build posres/refpos
    refpos_path,_ = box_cnf.write_refpos(out_prefix_path+".rfp")
    posresspec_path,_ = box_cnf.write_possrespec(out_prefix_path+".por", residues=list(box_cnf.residues.keys()))

    print(out_cnf_path)
    time.sleep(3)
    gromPP.frameout(in_top_path=out_top_path, in_coord_path=out_cnf_path, periodic_boundary_condition="v",
                out_file_path=out_prefix_path+"_box.pdb", out_file_format="pdb", time=0, verbose=True)

    #solvate
    out_solv_cnf_path=out_prefix_path+"_spc.cnf"
    gromPP.sim_box(in_top_path=out_top_path, minwall=None, in_cnf_path=out_cnf_path, in_solvent_cnf_file_path=solvents.spc,
                   out_cnf_path=out_solv_cnf_path, periodic_boundary_condition=None, gathering_method=None,  boxsize=True)
    time.sleep(2)

    gromPP.frameout(in_top_path=out_top_path, in_coord_path=out_solv_cnf_path, periodic_boundary_condition="r cog",
                    out_file_path=out_prefix_path+"_solv.pdb", include="all", out_file_format="pdb", time=0)


    # Emin solv

    emin_dir = out_dir+"/solv_emin"
    emin_outprefix = emin_dir+"/"+os.path.basename(out_dir)+"_emin"

    if(not os.path.exists(emin_dir)):
        os.mkdir(emin_dir)

    cnf_file = cnf.Cnf(out_solv_cnf_path)
    residues = cnf_file.get_residues()
    print(residues)
    import numpy as np
    t = list(map(lambda x: np.sum(list(x.values())) if(type(x) is dict) else x, residues.values()))
    print(t)
    all_atoms = np.sum(t)
    del residues["SOLV"]
    all_lig_atoms =np.sum(list(map(lambda x: np.sum(list(x.values())), residues.values())))
    in_por_path,_ = cnf_file.write_possrespec( out_path=emin_outprefix+".por", residues=list(residues.keys()))
    in_rpf_path,_ = cnf_file.write_refpos(emin_outprefix+".rpf")

    in_imd_path = emin_outprefix+".imd"
    imd_file = imd.Imd(emin_imd)

    imd_file.SYSTEM.NSM = (all_atoms - all_lig_atoms)//3

    pertubation_block  = imd_blocks.PERTURBATION(NTG=1, NRDGL=0,RLAM=0, DLAMT=0, ALPHLJ=0, ALPHC=0, NLAM=1, NSCALE=0)
    imd_file.add_block(block=pertubation_block)
    #imd_file.EDS = imd_blocks.EDS(NUMSTATES=2,S=1, EIR=[0 for x in range(2)])
    #multibath_block = imd_blocks.MULTIBATH(ALGORITHM=1, NBATHS=2, TEMP0=[298,298],
    #                                       TAU=[0.1,0.1], DOFSET=2,
    #                                       LAST=[all_lig_atoms, all_atoms], COMBATH=[1,2], IRBATH=[1,2])
    #imd_file.MULTIBATH = multibath_block

    imd_file.write(in_imd_path)

    gromXX.md_run(in_topo_path=out_top_path, in_imd_path=in_imd_path, in_coord_path=out_solv_cnf_path,
                  out_prefix=emin_outprefix, in_pert_topo_path=out_ptp_path,
                  in_refpos_path=in_rpf_path, in_posresspec_path=in_por_path)


[('M030', '_O71'), ['M030', 'F313'], ['M030', 'G277'], ['M030', 'E1VB'], ['6J29', 'G078'], ['M030', 'M097'], ['M030', 'G078'], ['M030', 'S002'], ('M030', '_O70'), ('M030', '_O6T'), ['M030', '6J29'], ['M030', '8018'], ['M030', 'TVVS'], ['M030', '6KET'], ['M030', 'G209'], ['M030', 'M218'], ['8018', 'G078'], ['8018', '6J29'], ('M030', '_P8I')]

M030__O71 ['M030', '_O71']
/home/bschroed/Documents/code/restraintmaker/devtools/otherScripts/b_ATB_solvationFreeEnergies/sets/pairwise/M030__O71/M030__O71.cnf
gromosPP.frameout: command:
/home/bschroed/Documents/code/gromosPP/installed/bin/frameout @topo /home/bschroed/Documents/code/restraintmaker/devtools/otherScripts/b_ATB_solvationFreeEnergies/sets/pairwise/M030__O71/M030__O71.top @traj /home/bschroed/Documents/code/restraintmaker/devtools/otherScripts/b_ATB_solvationFreeEnergies/sets/pairwise/M030__O71/M030__O71.cnf @pbc v  @outformat pdb @include SOLUTE @time 0 
STDOUT:  
/home/bschroed/Documents/code/gromosPP/installed/bin/sim_box @topo /ho

OSError: Could not find file: /home/bschroed/Documents/code/restraintmaker/devtools/otherScripts/b_ATB_solvationFreeEnergies/sets/FRAME_00001.pdb